## GBM

[Greedy Function Approximation: A Gradient Boosting Machine](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf) 论文真的很长呀。

只有多读几遍，然后实现才能豁然开朗。

『纸上得来终觉浅，绝知此事要躬行』


### 2 算法 Gradient_Boost

- Input: 
    - 给定样本集： $\{(x_i, y_i)\}_{i = 1}^{N}$
    - 可微损失函数：$L(y, F(x))$
    - 基学习器: $h_m(x)$
    - 迭代次数: $M$
- Algorithm:
    1. 初始化模型: $ F_0(x) = \underset{\rho}{\arg\min} \sum_{i = 1}^{N}L(y_i, \rho)$
    2. For m = 1 to M:
        1. 根据损失函数计算 "pseudo-responses"
            $$
            \tilde{y_i} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x)=F_{m-1}(x)} \quad \mbox{for } i=1,\ldots,N.
            $$
        2. 根据 $\{(x_i,\tilde{y_i})\}_{i=1}^n.$ 来训拟合出一个弱分类器 $h(x_i;\mathbf{a})$ 
        $$
        \mathbf{a}_m = \underset{\mathbf{a},\beta} {\arg\min} \sum_{i=1}^{N}[\tilde{y_i} - \beta h(x_i;\mathbf{a})]^2
        $$
        3. 计算出 $\rho_m$
        $$
        \rho_m = \underset{\rho}{\operatorname{arg\,min}} \sum_{i=1}^n L\left(y_i, F_{m-1}(x_i) + \rho h(x_i; \mathbf{a}_m)\right).
        $$
        4. 更新模型
        $$
        F_{m}(x)=F_{{m-1}}(x)+\rho_{m}h(x;\mathbf{a}_m).   
        $$
    3. 输出 $F_{M}(x)$

Gradient Boost 算法中的 $F_0, \tilde{y_i}, \rho_m$ 根据不同的损失函数  $L$ 其计算方法也是不同的。

下面讨论下，LSE, LAE, Huger，

### 3. LSE

LSE, Least Squared Error, 即 $L(y, F) =\frac{1}{2}(y - F)^2$.

1. $ F_0(X) = \bar{y} $
2. $ \tilde{y_i} = y_i - F_{m-1}(X_i)$
3. $\rho_m = \beta_m$，即训练与预测是一致的
    $$
    (\rho_m, \mathbf{a}_m) = \underset{\mathbf{a},\rho}{\arg\min} \sum_{i = 1}^{N}[\tilde{y_i} - \rho h(x_i;\mathbf{a})]^2
    $$
4. 模型迭代更新: $F_{m}(x)=F_{{m-1}}(x)+\rho_{m}h(x;\mathbf{a}_m).$

### 4. LAE

LSE, Least Absolute Error, 即 $L(y, F) =|y - F|$.

1. $ F_0(X) = \bar{y} $
2. $ \tilde{y_i} = sign(y_i - F_{m-1}(X_i))$
3. 使用 LAE 时训练与预测是**不一致**的，训练时使用的是残差符号，预测时需要用叶子节点上样本上的中位数(median)来预测。
    \begin{align}
    \rho_m &=     \underset{\rho}{\arg\min} \sum_{i = 1}^{N} L(y_i,  F_{m-1}(X_i) + \rho h(X_i;\mathbf{a}_m)) \\
    &= \underset{\rho}{\arg\min} \sum_{i = 1}^{N}|{y_i} - F_{m-1}(X_i) - \rho h(X_i;\mathbf{a}_m))|
    \end{align}
4. 模型迭代更新: $F_{m}(x)=F_{{m-1}}(x)+\rho_{m}h(x;\mathbf{a}_m).$

### 5. Regression Trees （GBDT, 用是 CART 是回归树）

回归树，考虑一个特殊情况，当基学习器是一个 J-叶节点回归树时，我们可以使用一个更通用的模型。

假设每一个回归树的模型为: 

$$
h(X;\{b_j, R_j\}_1^J) = \sum_{j=1}^{J}b_j1(X\in R_j)
$$

$\{R_j\}_1^J$ 是对目标值的一个划分， $1$ 是指示函数。则回归树的递归更新就变成了：
$$
F_m(X) = F_{m-1}(X) + \rho_m\sum_{j=1}^{J}b_{jm}1(X\in R_{jm})
$$

当使用 **LSE** 来拟合残差时，

$$
b_{jm} = ave_{x_i \in R_{jm}} \tilde{y_i}
$$

就对应回归树上叶子结点残差的均值。即更新公式可以写为：


$$
F_m(X) = F_{m-1}(X) + \sum_{j=1}^{J}\gamma_{jm}1(X\in R_{jm})
$$

其中：$\gamma_{jm} = \rho_m b_{jm}$


当使用 **LAE** 来拟合残差时，

$$
\gamma_{jm} = median_{x_i \in R_{jm}}\{y_i - F_{m-1}(X_i)\}
$$

就对应回归树上叶子上残差的中位数。

我们可以给出当使用 LAE 时提升树的算法：

<img src="./images/LAD.png" width="360px">

### 6. Huger 损失函数

### 7. Two-class LR and Classification

搞清楚上面对应的损失函数，对应残差的计算方法，模型更新方法，就可写出我们的模型了。